## Imports

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components
from datetime import date, timedelta, datetime

import os

## Load dataframe from newest file

In [4]:
for file in os.listdir('newest'):
    if "csv" in file:
        print("Loading "+file+" ...")

df_mx = pd.read_csv('./newest/'+file, encoding = "ISO-8859-1")
print(file+" Loaded")

Loading 201014COVID19MEXICO.csv ...
201014COVID19MEXICO.csv Loaded


## Active Cases of Mexico

In [12]:
df = df_mx

df = df.loc[(df['RESULTADO_LAB'] == 1)] # resultado = positivo SARS-COV-2 o Pendiente

# df = df = df.loc[df['ENTIDAD_RES'] == 2] #entidad = Baja California

# df = df.loc[df['MUNICIPIO_RES'] == 2] # municipio = mexicali

df = df[['FECHA_SINTOMAS','RESULTADO_LAB']]

df.loc[df['RESULTADO_LAB'] == 1, 'CONFIRMADO'] = ['1']

df_by_date = df.groupby("FECHA_SINTOMAS",as_index=False).count()
df_by_date['FECHA_SINTOMAS'] = pd.to_datetime(df_by_date['FECHA_SINTOMAS'])

first_case_date = df_by_date['FECHA_SINTOMAS'].iloc[0]

last_case_date = df_by_date['FECHA_SINTOMAS'].iloc[-1]

delta = (last_case_date) - first_case_date       # as timedelta

active_dict = {}
for i in range(delta.days + 1):
    day = first_case_date + timedelta(days=i)

    active_date_minus_14_days = day - timedelta(days=13)
    mask = (df_by_date['FECHA_SINTOMAS'] >= active_date_minus_14_days) & (df_by_date['FECHA_SINTOMAS'] <= day)
    filtered_df=df_by_date.loc[mask]
  
    df = filtered_df.sum(axis=0)
    active_dict[day] = df['CONFIRMADO']

active_df = pd.DataFrame(list(active_dict.items()),columns = ['FECHA_INICIO_SINTOMAS','ACTIVOS'])

fig = px.line(active_df, active_df.FECHA_INICIO_SINTOMAS, active_df.ACTIVOS, 
              title='Activos Mexico')
fig.show()